In [27]:
import pandas as pd
import itertools
import numpy as np
import torch
from torch_geometric.data import Data
import numpy as np


/Users/osulldan/miniforge3/envs/graphs/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:


# Download data (quietly)
!curl https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player.csv -o tbl_player.csv
!curl https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_player_skill.csv -o tbl_player_skill.csv
!curl https://raw.githubusercontent.com/batuhan-demirci/fifa21_dataset/master/data/tbl_team.csv -o tbl_team.csv



In [3]:
# Load data
player_df = pd.read_csv("tbl_player.csv")
skill_df = pd.read_csv("tbl_player_skill.csv")
team_df = pd.read_csv("tbl_team.csv")

# Extract subsets
player_df = player_df[["int_player_id", "str_player_name", "str_positions", "int_overall_rating", "int_team_id"]]
skill_df = skill_df[["int_player_id", "int_long_passing", "int_ball_control", "int_dribbling"]]
team_df = team_df[["int_team_id", "str_team_name", "int_overall"]]

# Merge data
player_df = player_df.merge(skill_df, on='int_player_id')
fifa_df = player_df.merge(team_df, on='int_team_id')

# Sort dataframe
fifa_df = fifa_df.sort_values(by="int_overall_rating", ascending=False)

# Sort to define the order of nodes
sorted_df = fifa_df.sort_values(by="int_player_id")
# Select node features
node_features = sorted_df[["str_positions", "int_long_passing", "int_ball_control", "int_dribbling"]]
# Convert non-numeric columns
pd.set_option('mode.chained_assignment', None)
positions = node_features["str_positions"].str.split(",", expand=True)
node_features["first_position"] = positions[0]
# One-hot encoding
node_features = pd.concat([node_features, pd.get_dummies(node_features["first_position"])], axis=1, join='inner')
node_features.drop(["str_positions", "first_position"], axis=1, inplace=True)



Players:  18767


,int_player_id,str_player_name,str_positions,int_overall_rating,int_team_id,int_long_passing,int_ball_control,int_dribbling,str_team_name,int_overall
0,1,Lionel Andrés Messi Cuccittini,"RW, ST, CF",93,5.0,91,96,96,FC Barcelona,84
33,2,Cristiano Ronaldo dos Santos Aveiro,"ST, LW",92,6.0,77,92,88,Juventus,83
57,3,Jan Oblak,GK,91,8.0,40,30,12,Atlético Madrid,83
121,5,Neymar da Silva Santos Júnior,"LW, CAM",91,7.0,81,95,95,Paris Saint-Germain,83
89,4,Kevin De Bruyne,"CAM, CM",91,2.0,93,92,88,Manchester City,85


In [10]:
# Convert to numpy
x = node_features.to_numpy()
x.shape # [num_nodes x num_features]

# Select node features
labels = sorted_df[["int_overall"]]
labels.head()

y = labels.to_numpy()
y.shape

fifa_df["int_player_id"] = fifa_df.reset_index().index


teams = fifa_df["str_team_name"].unique()
all_edges = np.array([], dtype=np.int32).reshape((0, 2))


for team in teams:
    team_df = fifa_df[fifa_df["str_team_name"] == team]
    players = team_df["int_player_id"].values
    # Build all combinations, as all players are connected
    permutations = list(itertools.combinations(players, 2))
    edges_source = [e[0] for e in permutations]
    edges_target = [e[1] for e in permutations]
    team_edges = np.column_stack([edges_source, edges_target])
    all_edges = np.vstack([all_edges, team_edges])



# Convert to Pytorch Geometric format
edge_index = all_edges.transpose()
edge_index # [2, num_edges]

(18767, 1)

In [22]:
np.shape(all_edges)
e = torch.tensor(edge_index, dtype=torch.long)
edge_index1 = e.t().clone().detach()



print (np.shape(y))
print (np.shape(x))
print (np.shape(all_edges))

NameError: name 'torch' is not defined

In [37]:
data = Data(x=x, edge_index=edge_index1.to().contiguous(), y=y)

In [28]:
help (Data)


Help on class Data in module torch_geometric.data.data:

class Data(BaseData, torch_geometric.data.feature_store.FeatureStore, torch_geometric.data.graph_store.GraphStore)
 |  Data(x: Optional[torch.Tensor] = None, edge_index: Optional[torch.Tensor] = None, edge_attr: Optional[torch.Tensor] = None, y: Optional[torch.Tensor] = None, pos: Optional[torch.Tensor] = None, **kwargs)
 |
 |  A data object describing a homogeneous graph.
 |  The data object can hold node-level, link-level and graph-level attributes.
 |  In general, :class:`~torch_geometric.data.Data` tries to mimic the
 |  behavior of a regular Python dictionary.
 |  In addition, it provides useful functionality for analyzing graph
 |  structures, and provides basic PyTorch tensor functionalities.
 |  See `here <https://pytorch-geometric.readthedocs.io/en/latest/get_started/
 |  introduction.html#data-handling-of-graphs>`__ for the accompanying
 |  tutorial.
 |
 |  .. code-block:: python
 |
 |      from torch_geometric.data imp

In [34]:
e = torch.tensor(edge_index, dtype=torch.long)
print(e)


edge_index1 = e.t().clone().detach()
edge_index1


tensor([[    0,     0,     0,  ..., 18704, 18704, 18719],
        [    7,    32,    45,  ..., 18719, 18751, 18751]])


tensor([[    0,     7],
        [    0,    32],
        [    0,    45],
        ...,
        [18704, 18719],
        [18704, 18751],
        [18719, 18751]])